# Исследование надежности заемщиков.
Используя данные банка, необходимо выяснить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

### Шаг 1. Обзор данных
Данные - статистика банка о платежеспособности клиентов.<br>
•	children — количество детей в семье<br>
•	days_employed — общий трудовой стаж в днях<br>
•	dob_years — возраст клиента в годах<br>
•	education — уровень образования клиента<br>
•	education_id — идентификатор уровня образования<br>
•	family_status — семейное положение<br>
•	family_status_id — идентификатор семейного положения<br>
•	gender — пол клиента<br>
•	income_type — тип занятости<br>
•	debt — имел ли задолженность по возврату кредитов<br>
•	total_income — ежемесячный доход<br>
•	purpose — цель получения кредита<br>

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/datasets/data.csv')
print(data.info())
print()
data.head() #Знакомимся с данными.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Сразу можно отметить, что в данных присутствуют некоторые проблемы: пропуски в столбцах 'days_employed' и 'total_income', разные регистры в столбце 'education'.

### Шаг 2.1 Заполнение пропусков
В данных есть пропуски в двух столбцах: 'days_employed' и 'total_income'. Пропуски составляют 11% от всех значений столбца. Это много, соответстенно, данные с пропусками нельзя просто удалить. Количество пропусков в обоих столбцах совпадает. Скорее всего, это строки безработных клиентов, тогда логично, что данные о трудовом стаже и о доходе отсутсвуют. Пропуски в данных о доходах заполним медианным значением по столбцу, так как медианное значение лучше отражает средний показатель и не зависит от выделяющихся крупных и мелких значений, как среднее арифметическое.

In [3]:
data.isna().sum() #Ищем пропущенные значения
none_share = data['total_income'].isna().sum() / data['total_income'].count() * 100#Находим долю пропущенных значений в столбце.
#print(none_share)
total_income_median = data['total_income'].median() #Находим медианное значение столбца.
data['total_income'] = data['total_income'].fillna(total_income_median) #Заполняем пропуски в столбце медианным значением.

### Шаг 2.2 Проверка данных на аномалии и исправления.
Столбец 'days_employed' содержит отрицательные значения. Количество дней трудового стажа не может быть отрицательным, соответственно, можно сделать вывод, что это механическая ошибка. Чтобы заполнить пропуски в данном столбце, необходимо сначала избавиться от отрицательных значений, а затем посчитать медианное значение по столбцу и заполнить им пропуски. Столбец 'children' тоже содержит отрицательное значение, что невозможно, избавимся от отрицательных значений и в этом столбце.

In [4]:
data['days_employed'] = data['days_employed'].abs() #Избавимся от отрицательных значений в столбце.
days_employed_median = data['days_employed'].median() #Находим медианное значение столбца.
data['days_employed'] = data['days_employed'].fillna(days_employed_median) #Заполняем пропуски в столбце медианным значением.
#data.isna().sum() #Проверим, не осталось ли пропусков в данных.
data['children'] = data['children'].abs() #Избавимся от отрицательных значений в столбце.

Среди значений в столбце 'children' сильно выделяется значение 20. Скорее всего, это тоже ошибка, заменим 20 на 2.

In [5]:
#data['children'].unique() #Проверяем значения в столбце.
data.loc[data['children']==20, 'children'] = 2 #Заменяем значения в столбце на основе условия.

### Шаг 2.3. Изменение типов данных.
Для удобства анализа заменим дробные числа в столбцах 'days_employed' и 'total_income' на целые.

In [6]:
data['days_employed'] = data['days_employed'].astype(int) #Заменяем вещественный тип данных в столбце на целочисленный.
data['total_income'] = data['total_income'].astype(int) #Заменяем вещественный тип данных в столбце на целочисленный.

### Шаг 2.4. Удаление дубликатов.
Данные содержат строки-дубликаты, их необходимо удалить. Также в двух столбцах ('education', 'purpose') содержатся неявные дубликаты, от которых тоже необходимо избавиться. Дубликаты в столбце 'education' связаны с тем, что при введении данных был использован разный регистр, удалить эти дубликаты можно, приведя все значения столбца к единому регистру. Неявные дубликаты в столбце 'purpose' связаны с тем, что при заполнении данного столбца использовались синонимы и различные формулировки для одних и тех же целей. Для работы с этим столбцом в дальнейшем необходимо будет разделить все цели на категории и создать для этих категорий отдельный столбец.

In [7]:
data.duplicated().sum() #Проверяем, есть ли в данных дубликаты.
data = data.drop_duplicates().reset_index(drop=True) #Удаляем строки-дубликаты.
data['education'].unique() #Проверяем столбец на наличие неявных дубликатов.
data['education_id'].unique() #Проверяем столбец на наличие неявных дубликатов.
data['family_status'].unique() #Проверяем столбец на наличие неявных дубликатов.
data['family_status_id'].unique() #Проверяем столбец на наличие неявных дубликатов.
data['gender'].unique() #Проверяем столбец на наличие неявных дубликатов.
data['income_type'].unique() #Проверяем столбец на наличие неявных дубликатов.
data['debt'].unique() #Проверяем столбец на наличие неявных дубликатов.
data['purpose'].unique() #Проверяем столбец на наличие неявных дубликатов.
data['education'] = data['education'].str.lower() #Приведем все значения столбца к нижнему регистру.
#data['education'].unique() #Проверим избавление от неявных дубликатов в столбце.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.
Для того, чтобы сделать данные более компактными и удобными для анализа, необходимо удалить столбцы 'education' и 'family_status', предварительно вынеся их в новые датафреймы, к которым можно будет обращаться по идентификатору.

In [8]:
data_education = data[['education','education_id']] #Создаем новый датафрейм.
data_family = data[['family_status', 'family_status_id']] #Создаем новый датафрейм.
data = data.drop(['education'], axis=1) #Удаляем столбец из датафрейма.
data = data.drop(['family_status'], axis=1) #Удаляем столбец из датафрейма.

### Шаг 2.6. Категоризация дохода.
Для удобства анализа разделим всех клиентов на категорию по уровню дохода и создадим отдельных столбец с категориями. <br>
0–30000 — 'E';<br>
30001–50000 — 'D';<br>
50001–200000 — 'C';<br>
200001–1000000 — 'B';<br>
1000001 и выше — 'A'.

In [9]:
def total_income_category(total_income): #Функция для деления на категории по уровню дохода.
    if 0 < total_income < 30000:
        return 'E'
    if 30001 < total_income < 50000:
        return 'D'
    if 50001 < total_income < 200000:
        return 'C'
    if 200001 < total_income < 1000000:
        return 'B'
    if total_income > 1000001:
        return 'A'
data['total_income_category'] = data['total_income'].apply(total_income_category) #Создаем новый столбец с категориями клиентов по уровню дохода.

### Шаг 2.7. Категоризация целей кредита.
Для удобства анализа разделим всех клиентов на категории по целям, на которые был взят кредит.<br>
'операции с автомобилем',<br>
'операции с недвижимостью',<br>
'проведение свадьбы',<br>
'получение образования'.

In [10]:
#data['purpose'].unique() #Посмотрим, какие формулировки целей есть в столбце.
def purpose_category(purpose): #Функция для деления на категории по целям кредита.
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'недвиж' in purpose:
        return 'операции с недвижимостью'
    if 'свадь' in purpose:
        return 'проведение свадьбы'
    if 'образ' in purpose:
        return 'получение образования'
data['purpose_category'] = data['purpose'].apply(purpose_category) #Создаем новый столбец с категориями клиентов по целям кредита.

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [11]:
pd.pivot_table(data, index='children', values='debt')

,debt
children,
0,0.075353
1,0.091639
2,0.094925
3,0.081818
4,0.097561
5,0.000000


##### Вывод 1:
По полученной таблице видно, что количество должников в семьях с разным количеством детей примерно одинаковое - от 7 до 10%. Выбивается только показатель должников среди семей с 5 детьмя - 0%, но это должно быть связано с тем, что семей с 5 детьмя в предоставленных данных меньше всех, поэтому, показатель по клиентам с 5 детьми можно считать ненадежным.. Таким образом, можно сделать вывод, что количество детей не влияет критично на возврат кредита в срок, но самыми надежными можно назвать клиентов без детей, а самыми ненадежными - клиентов с 4 детьми.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [12]:
print(pd.pivot_table(data, index=['family_status_id'], values=['debt'])) #Строим сводную таблицу для сравнения.

                      debt
family_status_id          
0                 0.075421
1                 0.093202
2                 0.065693
3                 0.071130
4                 0.097509


##### Вывод 2:
По полученной таблице видно, что возврат кредита в срок не зависит критично от гражданского статуса кредитора, количество должников по всем категориям семейного статуса остается в диапазоне 7-10%. Однако, можно сделать вывод, что самыми надежными клиентами являются вдовы и вдовцы, а самыми ненадежными - холостые клиенты.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [13]:
pd.pivot_table(data, index=['total_income_category'], values=['debt']) #Строим сводную таблицу для сравнения.

,debt
total_income_category,
A,0.080000
B,0.070635
C,0.084825
D,0.060000
E,0.090909


##### Вывод 3:
По полученной таблице видно, что наибольшее количество должников - среди клиентов с самым низким уровнем дохода, а самый низкий процент - среди клиентов с уровнем дохода ниже среднего. Однако, процент должников в каждой группе различается очень слабо, все данные входят в диапазон от 6 до 9%.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [14]:
pd.pivot_table(data, index=['purpose_category'], values=['debt']) #Строим сводную таблицу для сравнения.

,debt
purpose_category,
операции с автомобилем,0.093547
операции с недвижимостью,0.072314
получение образования,0.092177
проведение свадьбы,0.079657


##### Вывод 4:
По полученной таблице, как и по предыдущим, видно, что процент должников в группах отличается минимально (7-9%). Таким образом, можно сделать вывод, что цель кредита сильно не влияет на его возврат в срок. Однако, можно сделать вывод, что кредиты на недвижимость и свадьбы чаще выплачиваются без просрочек, чем кредиты на автомобили и образование.

## Общий вывод:

В ходе исследования были проанализированы данные о платежеспособности клиентов банка. Было проверено, как влияют на платежеспособность клиента гражданский статус, количество детей в семье, уровень дохода и цели кредита. Были получены результаты, согласно которым ни один из этих показателей не влияет на возврат клиентом кредита в срок критично. Однако, некоторые различия все-таки есть. Так, клиент без детей с большей вероятностью вернет кредит в срок, чем многодетный клиент. В то же время холостые клиенты не могут считаться надежными по сравнению с вдовцами и разведенными клиентами. Также самыми надежными можно счиать клиентов с уровнем дохода ниже среднего, котоые берут кредит с целью покупки недвижимости.